# Fanocodierung
Für dieses Praktikum verwenden wir die Klassen für `BitArray`, `BSC` und `CRC` aus dein ersten beiden Praktika.

In [1]:
from math import ceil


class BitArray:
    def __init__(self, bits=None, length=None, file=None):
        if bits is None:
            self.array = 0
            self.length = 0
            if file is not None:
                self.write(file)
        elif type(bits) in {chr, str}:
            self.array = int(bits, 2)
            self.length = len(bits)
            if file is not None:
                self.write(file)
        elif type(bits) == int:
            self.array = bits
            self.length = length if length else bits.bit_length()
            if file is not None:
                self.write(file)
        elif type(bits) == BitArray:
            self.array = bits.array
            self.length = bits.length
            if file is not None:
                self.write(file)
        elif type(bits) == bytes:
            temp = BitArray(int.from_bytes(bits, 'big'))
            cutoff = int(temp[:8])
            self.array = int(temp[8 + cutoff:])
            self.length = len(temp) - (8 + cutoff)
            del temp
            if file is not None:
                self.write(file)
        elif bits is None and file is not None:
            try:
                self.__init__(self.read(file))
            except:
                raise Exception(f'File {file} is invalid!')
        else:
            raise Exception(f"""
Bits must be of type str or int.
{type(bits)} is something completly different.
""")
    
    def __call__(self, step=1):
        for i in range(0, len(self), step):
            if i+step <= len(self):
                yield self[i:i+step]
    
    def __iter__(self):
        for i in range(len(self)):
            yield self[i]
    
    def __getitem__(self, idx):
        if type(idx) == slice:
            start = (idx.start if idx.start is not None else 0) % len(self)
            stop = (idx.stop if idx.stop is not None else len(self)) % (len(self) + 1)
            return BitArray((self.array >> len(self) - stop) & ((1 << stop - start) - 1),
                            length=stop-start)
        else:
            return BitArray((self.array & 1 << (len(self) - idx - 1)) >> (len(self) - idx - 1) , length=1)
    
    def __setitem__(self, idx, value):
        if type(value) != BitArray:
            raise Exception(f"""
Value must be of type BitArray.
{type(value)} is something completly different.
""")
        cutoff = min(len(self), idx + len(value))
        self.array = (self[:idx]|value[:cutoff]|self[cutoff:]).array
    
    def __hash__(self):
        return hash(str(self))
    
    def __or__(self, other):
        return BitArray(self.array << len(other)|other.array, length=len(self) + len(other))
    
    def __xor__(self, other):
        if isinstance(other, BitArray):
            return BitArray(self.array ^ other.array, length=max(len(self), len(other)))
        else:
            return BitArray(self.array ^ other, length=max(len(self), other.bit_length()))
    
    def __repr__(self):
        return f'BitArray[{str(self)}]'
    
    def __str__(self):
        if self.array is not None:
            return bin(self.array).replace('0b', '').zfill(len(self))[:len(self)]
        else:
            return ''
    
    def bytes(self):
        return (BitArray(-len(self) % 8, length=8)|self).array.to_bytes(ceil(len(self)/8) + 1, 'big')
    
    def __len__(self):
        return self.length
    
    def __int__(self):
        return self.array
    
    def __bool__(self):
        return bool(self.array)
    
    def __eq__(self, other):
        return self.array == other.array and len(self) == len(other)
    
    def __lshift__(self, idx):
        return BitArray(self.array << idx, length=self.length + idx)
    
    def __rshift__(self, idx):
        return BitArray(self.array >> idx, length=max(self.length - idx, 0))


def concatenate(args):
    result = BitArray()
    for arg in args:
        result = result | BitArray(arg)
    return result

In [2]:
from random import uniform


class BSC:
    def __init__(self, p):
        self.p = p
    
    def error(self, n):
        return BitArray(''.join('0' if uniform(0, 1) > self.p else '1' for _ in range(n)))
    
    def __call__(self, m):
        return self.error(len(m)) ^ m

In [3]:
class CRC:
    def __init__(self, g, k):
        self.g = g
        self.k = k
    
    def add(self, a, b):
        return a ^ b
    
    def mul(self, a, b):
        result = BitArray()
        for i, v in enumerate(a):
            i = len(a) - i - 1
            result = result ^ (int(v) * int(b) << i)
        return result
    
    def mod(self, a):
        result = BitArray()
        rest = BitArray(a)
        for i, v in enumerate(rest[:-len(self.g)]):
            result <<= 1
            result = result ^ v
            rest = rest ^ (int(rest[i]) * int(self.g) << (len(a) - len(self.g) - i))
        rest.length = self.g.length - 1
        return rest

    def p(self, s):
        return self.mod(s<<(len(self.g)-1))
    
    def c(self, s):
        return self.add(self.p(s), s<<(len(self.g)-1))
    
    def secure(self, stream):
        output = BitArray()
        for s in stream(self.k):
            output = output|self.c(s)
        return output
    
    def check(self, stream):
        return all(not self.mod(c) for c in stream(self.k + len(self.g) - 1)) 

## Häufigkeitsanalyse

Beginnen wir nun mit dem tatsächlichen Praktikum. Zu aller erst wollen wir eine Funktion erstellen, welche für eine gegebene Datei eine Häufigkeitsanalyse (FA, frequency analysis) erstellt. Für diese haben wir folgende Möglichkeiten ein Alphabet zu wählen:

1. Wir wählen ein predeterminiertes Alphabet aus. Beispielsweise indem wir uns von anfang an auf Ascii festlegen und alle Buchstaben die nicht darunter fallen ignorieren.
2. Wir wählen ein Alphabet wie ISO-XXXX oder UTF-X adaptiv, indem wir schauen, welches der Alphabete alle Buchstaben unserer Datei beinhaltet.
3. Wir erstellen unser eigenes Alphabet aus den Zeichen, welche in der Datei vorhanden sind.

Letztere Option scheint die effizienteste und am leichtesten zu implementierende zu sein. Sie birgt jedoch den Nachteil, dass Mögliche *artverwandte* aber nicht im Text vorhandene Buchstaben garnicht erst aufführt. Wird z.B. eine Textdatei die nur aus Kleinbuchstaben besteht erfasst, so finden sich keine Daten über Großbuchstaben in unserer FA obwohl diese auf Grund ihrer *Verwandheit* zu Kleinbuchstaben dort erwartet werden könnten. Sollte also ein Programm mit der erstellten Häufigkeitsanalyse arbeiten und auf einen Buchstaben zugreifen wollen, der nicht vorhanden ist, so werden sich einige NullptrExceptions anhäufen. In unserem Falle wird eine FA nur mit dem Text verwendet, von dem sie erstellt wurde, womit wir diese Problematik umgehen können.

Zur Erinnerung:
$$H(X) := - \sum_{i=1}^{|X|} \text{ld}(P(x_i))P(x_i)$$

In [4]:
from math import log2 as ld, ceil


I = lambda p: -ld(p)
H = lambda ps: sum(I(p) * p for p in ps)


def frequency_analysis(file_name):
    with open(file_name, 'r') as file:
        data = file.read()
        frequency = {char: data.count(char)/len(data) for char in set(data)}
    return frequency


encoding_file_name = 'hello_world.txt'#'Burrows Wheeler Transformation.md'
decoding_file_name = encoding_file_name.replace('.', ' (decoded).')


frequencies = frequency_analysis(encoding_file_name)
print(f"""
File entropy: {(e := H(frequencies.values()))}
Bit minimum:  {(m := ceil(len(open(encoding_file_name, 'r').read()) * e))}
""")
for key, frequency in frequencies.items():
    print(f'Char: {repr(key)},\tFrequency: {round(frequency * 100, 2)}%,\tInformation: {round(I(frequency), 2)}[bit]')


File entropy: 3.0220552088742005
Bit minimum:  37

Char: 'W',	Frequency: 8.33%,	Information: 3.58[bit]
Char: 'r',	Frequency: 8.33%,	Information: 3.58[bit]
Char: 'o',	Frequency: 16.67%,	Information: 2.58[bit]
Char: ' ',	Frequency: 8.33%,	Information: 3.58[bit]
Char: 'H',	Frequency: 8.33%,	Information: 3.58[bit]
Char: '\n',	Frequency: 8.33%,	Information: 3.58[bit]
Char: 'l',	Frequency: 25.0%,	Information: 2.0[bit]
Char: 'e',	Frequency: 8.33%,	Information: 3.58[bit]
Char: 'd',	Frequency: 8.33%,	Information: 3.58[bit]


## Fanocode

Die Parentfunktion dieser Implementierung des FanoCodes `FanoCodeBook` erstellt eine sortierte Liste aus Tupeln, welche jeweils einen Buchstaben und dessen Häufigkeit beinhalten. Hier werden lediglich Datenstrukturen gewechselt.
`FanoRecursive` erstellt ein Codebuch durch Aufteilung der sortierten Liste in möglichst gleichgroße Hälften, welche rekursive zu zusammen gesetzt werden.

## Codebuch Encoding
Da wir das Codebuch mitverschicken wollen müssen wir es vorab codieren. Dies wird von der `CodeBookEncode` funktion getan. Das codierte Codebuch wird als Prefix vor dem Codierten Fanocode gesendet und zur Dekodierung abgetrennt und wieder rekonstruiert. So ist lediglich der Bitstream zu vollständigen Decodierung notwendig.

In [5]:
def CodeBookEncode(codebook):
    char_len = max(ord(char).bit_length() for char in codebook.keys())
    freq_len = max(len(freq) for freq in codebook.values())
    block = concatenate(BitArray(len(val), length=freq_len)|BitArray(ord(key), length=char_len)|BitArray(val) for key, val in codebook.items())
    return BitArray(len(codebook), length=16)|BitArray(char_len, length=8)|BitArray(freq_len, length=8)|block


def CodeBookDecode(stream):
    block_num = int(stream[:16])
    char_len = int(stream[16:24])
    freq_len = int(stream[24:32])
    codebook = dict()
    idx = 32
    for _ in range(block_num):
        val_len = int(stream[idx:idx+freq_len])
        char = chr(int(stream[idx+freq_len:idx+freq_len+char_len]))
        val = str(stream[idx+freq_len+char_len:idx+freq_len+char_len+val_len])
        codebook[char] = val
        idx += freq_len + char_len + val_len
    return codebook, stream[idx:]


def FanoCodeBook(frequencies):
    frequencies = sorted(list(frequencies.items()), key=lambda x: x[1], reverse=True)
    return FanoRecursive(frequencies)


def FanoRecursive(frequencies):
    if len(frequencies) == 1:
        return {frequencies[0][0]: ''}
    half_sum, max_sum = 0, sum(x for _, x in frequencies)
    for idx, (_, frequency) in enumerate(frequencies):
        if (half_sum + frequency) * 2 > max_sum:
            if (half_sum + frequency) * 2 - max_sum > max_sum - half_sum * 2:
                cut_off = idx
            else:
                cut_off = idx + 1
            result = {key: '0' + val for key, val in FanoRecursive(frequencies[:cut_off]).items()}
            result.update({key: '1' + val for key, val in FanoRecursive(frequencies[cut_off:]).items()})
            return result
        half_sum += frequency
        


def FanoEncode(file_name):
    frequencies = frequency_analysis(file_name)
    codebook = FanoCodeBook(frequencies)
    print(codebook)
    with open(file_name, 'r') as file:
        data = file.read()
        result = BitArray()
        for char in data:
            result = result | BitArray(codebook[char])
        return CodeBookEncode(codebook)|result

    
def FanoDecode(stream):
    codebook, stream = CodeBookDecode(stream)
    codebook = {val: key for key, val in codebook.items()}
    start, end, result = 0, 0, []
    while end < len(stream):
        if (bits := str(stream[start:end])) in codebook:
            result.append(codebook[bits])
            start = end
        else:
            end += 1
    return ''.join(result)

## Simulation

Im Folgenden durchläuft die Datei eine Simulation zur Übertragung. Hierbei werden zu Anfang gewisse Parameter für das Transmissions Protokoll festgelegt.

|Variable|Beschreibung|
|---|---|
|`block_size`| Länge der zu verschickenden bereits quellencodierten Blöcke|
|`p`|Fehlerwahrscheinlichkeit pro Bit|
|`CRC_8`| Bluetooth CRC Polynom|
|`package_size`|Länge der Blöcke nach Codierung mittels CRC|

`transmission` immitiert ein kleines Transmissionsprotokoll. Wird ein Paket fehlerhaft erhalten so muss es erneut gesended werden. `padding` ermöglicht die Veränderung der Länge der zu sendenden Daten auf ein vielfaches von `package_size`; `padding_inv` reversiert diesen Vorgang.

In [6]:
block_size = 24
p = 1e-4
channel = BSC(p)
CRC_8 = BitArray('111010101') # Bluetooth
GaloisField = CRC(CRC_8, block_size)
package_size = block_size + len(CRC_8) -1


def transmission(stream): # TCP style transmission 
    for block in stream(package_size):
        while not GaloisField.check((transmitted := channel(block))):
            print(f'Error ocurred in {transmitted}')
        yield transmitted

        
def padding(stream):
    offset = -(8 + len(stream)) % block_size
    return BitArray(bits=offset, length=8) | stream | BitArray('0'*offset)


def padding_inv(stream):
    offset = int(stream[:8])
    return stream[8:len(stream)-offset]


# Simulation startet
fano_encoding = FanoEncode(encoding_file_name)
padded_encoding = padding(fano_encoding)
crc_encoding = GaloisField.secure(padded_encoding)
crc_decoding = concatenate(block[:block_size] for block in transmission(crc_encoding))
unpadded_decoding = padding_inv(crc_decoding)
decoding = FanoDecode(unpadded_decoding)
# Simulation endet


with open(decoding_file_name, 'w') as file:
    file.write(decoding+'\n')

{'l': '00', 'o': '010', 'W': '011', 'r': '1000', ' ': '1001', 'H': '101', '\n': '1100', 'e': '1101', 'd': '111'}


## Test
Wir testen nun ob die decodierte Datei der originalen Datei gleicht. Das ausbleiben eines `AssertionErrors` zeigt das Bestehen des Tests.

In [7]:
original = open(encoding_file_name, 'r').read()
decoding = open(decoding_file_name, 'r').read()
assert original == decoding

## Kompressionsrate

Um die Kompressionsrate $K$ zu berechnen, teilen wir die Anzahl $|\_|$ der Bits nach der Codierung $C(x)$ durch die Anzahl der Bits vor der Codierung $|x|$.
$$K = \frac{|C(x)|}{|x|}$$

In [8]:
import os
c_x = len(fano_encoding)
x = (os.path.getsize(encoding_file_name) * 8 )
c = c_x/x

print(f"""
Compression um {round(c*100, 2)}%
Redundanz von  {round((len(fano_encoding)/m - 1)*100, 2)}%
""")


Compression um 206.25%
Redundanz von  435.14%

